# Tarefa 02 Módulo 05

O nosso projeto desta sequência de módulos do curso será um aprofundamento da demonstração sobre classificação de risco de crédito que vimos lá no comecinho. Pois recebemos uma base já montada pra nós. Tenha certeza de que ela passou por um longo processamento até ficar daquele jeito. Neste exercício vamos exercitar o que aprendemos nas ultimas aulas e montar a variável resposta da base do nosso projeto.

#### Marcação de bom e mau
O objetivo da modelagem é classificar o risco de inadimplência, ou como se diz no meio, o risco de *default*. Podemos fazer longas discussões sobre o conceito de *default* com base em estudos e exigências regulatórias, para efeitos deste estudo, um cliente em *default* é aquele que está em 60 dias de atraso ou mais. Então classificaremos os clientes como 'bons' e 'maus' assim:
- **Maus** pagadores: são aqueles que entraram em 'default' (atraso 60 dias ou mais) nos 24 meses seguintes à aquisição do cartão de crédito. 
- **Bons** pagadores: são considerados todos os demais.
- **Excluídos**: Clientes que não adquiriram um cartão de crédito (seja por recusa, seja por desistência) não possuem informações de pagamento, portanto não se pode identificar se são bons ou maus. Há uma longa discussão e literatura sobre *inferência de rejeitados* que está fora do escopo deste exercício.

#### Bases disponíveis
Temos duas bases importantes aqui: uma de propostas, com diversas informações dos vários solicitantes de cartão de crédito, e uma base de pagamentos. A base de pagamentos será utilizada para identificar a ocorrência de *default*. A base de propostas tem diversas informações coletadas no momento da solicitação do crédito (isto é importante: qualquer informação posterior a essa data é impossível de ser coletada na aplicação do modelo e não pode ser utilizada).

As variáveis delas são:

Base de propostas - application_records.csv

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| CODE_GENDER| M = 'Masculino'; F = 'Feminino' |M/F|
| FLAG_OWN_CAR| Y = 'possui'; N = 'não possui' |Y/N|
| FLAG_OWN_REALTY| Y = 'possui'; N = 'não possui' |Y/N|
| CNT_CHILDREN| Quantidade de filhos |inteiro|
| AMT_INCOME_TOTAL| Annual income |inteiro|
| NAME_INCOME_TYPE|Tipo de renda (ex: assaliariado, autônomo etc) | texto |
| NAME_EDUCATION_TYPE| Nível de educação (ex: secundário, superior etc) |texto|
| NAME_FAMILY_STATUS | Estado civil (ex: solteiro, casado etc)| texto |
| NAME_HOUSING_TYPE | tipo de residência (ex: casa/apartamento, com os pais etc) | texto |
| DAYS_BIRTH | Count backwards from current day (0), -1 means yesterday |inteiro|
| DAYS_EMPLOYED | Count backwards from current day (0), -1 means yesterday |inteiro|
| FLAG_MOBIL | Indica se possui celular (1 = sim, 0 = não) |binária|
| FLAG_WORK_PHONE | Indica se possui telefone comercial (1 = sim, 0 = não) |binária|
| FLAG_PHONE | Indica se possui telefone (1 = sim, 0 = não) |binária|
| FLAG_EMAIL | Indica se possui e-mail (1 = sim, 0 = não) |binária|
| OCCUPATION_TYPE | Occupation	 |Qualitativa|
| CNT_FAM_MEMBERS | quantidade de pessoas na residência |inteiro|

Base de pagamentos - pagamentos_largo.csv  

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| mes_00 a mes_24| faixa de atraso mês a mês do cliente <br>0: 1-29 days past due &nbsp;&nbsp;&nbsp;&nbsp; 1: 30-59 days past due <br />2: 60-89 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 3: 90-119 days overdue <br /> 4: 120-149 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 5: more than 150 days <br />C: paid off that month &nbsp;&nbsp;&nbsp;&nbsp; X: No loan for the month |Qualitativa|

#### Construindo a variável resposta
A base de pagamentos está em um formato de 'base larga'. Essa base possui informações de pagamentos do cliente mês a mês a partir do mês de aquisição do crédito (mês 0) até o vigésimo quarto mês após a aquisição do crédito (mês 24). Utilizaremos essa base para determinar se um proponente é considerado 'bom pagador' ou caso apresente atraso representativo, será considerado 'mau pagador'.

#### Base larga vs base longa
A base ser larga significa que há uma linha para cada cliente, e que as informações estarão nas colunas, em contraste com a 'base longa', em que haveria uma linha para cada combinação cliente/mês, uma coluna indicando o cliente, outra indicando o mês, e apenas uma coluna com a informação do atraso.

#### Tarefa 1) Marcar *default* no mês
Faça uma indicadora de se o cliente está em *default* em cada uma das marcações (mes_00 a mes_24). Dica: você pode utilizar o método ```.isin()``` do Pandas. Consulte a [documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html) caso necessário.

#### Tarefa 2) 'bons' e 'maus' ao longo de todos os 24 meses de desempenho
Marque para cada cliente se ele teve pelo menos um episódio de *default* entre o mês 0 e o mês 24. Dica: o método ```sum()``` pode ajudar. Caso precise, consulte a [documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html) e procure pelo argumento ```axis```, você viu outros métodos que possuem esse argumento também. Tendo o número de meses em default de cada cliente, basta marcar ```True``` para todos aqueles que possuem pelo menos 1 mês em *default* e ```False``` para os demais.

#### Tarefa 3) Marcando proponentes expostos ao risco de crédito
Marcando proponentes que se tornaram tomadores: lembre-se de que clientes que não adquiriram o cartão devem ser desconsiderados. A base de pagamentos possui apenas clientes que adquiriram cartão de crédito, então você pode selecionar somente os clientes da base de propostas que se encontram na base de pagamentos.

#### Tarefa 4) Consolidando as informações
Faça uma junção das informações da base de propostas com a variável de *default* que você acabou de construir. Talvez você consiga realizar a tarefa 3 e tarefa 4 em uma única linha de código ;)

#### Tarefa 5) Verificando
Faça uma contagem dos valores do *default* que você construiu. 

In [5]:
import pandas as pd
import numpy as np

In [6]:
propostas = pd.read_csv('application_record.csv')
pg = pd.read_csv('pagamentos_largo.csv')

In [25]:
pg.shape # vai do cliente 0 ao cliente 20936, totalizando 20937 clientes

(20937, 26)

In [ ]:
propostas = propostas.set_index('ID')
# Percebi que a variável 'ID' é um int, log, estava entrando no sum(), o que alterava significativamente os resultados

In [75]:
propostas.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
ID,,,,,,,,,,,,,,,,,
5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [76]:
pg = pg.set_index('ID')
pg.head()
#o mesmo procedimento com a variável 'ID' foi realizado para essa df

,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,mes_17,...,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
ID,,,,,,,,,,,,,,,,,,,,,
5001718,0,0,0,0,0,0,0,0,NaN,0,...,NaN,0,NaN,0,0,0,0,NaN,0,NaN
5001719,0,0,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
5001720,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN
5001726,0,0,C,C,C,C,C,C,C,C,...,C,C,C,0,0,0,C,C,C,C


In [77]:
# 1) Seu código aqui - Faça uma indicadora de se o cliente está em default em cada uma das marcações (mes_00 a mes_24). Dica: você pode utilizar o método .isin()

# pg = pg.fillna() - após refletir, entendi que substituir os NAN por zero ou qualquer outro número atrpalhará o somatório final

pg_indicadora_default = pg.isin(['C',0, 1]) # identificando com true e false a situação de default de cada cliente.
pg_indicadora_default.head()                             # o objetivo é marcar com True quem está pagando corretamente e False se ocorreu atraso.

# a pg_indicadora_default parece corresponder aos dados observados na pg original, portanto os dados tem alta probabilidade de serem consistentes.

,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,mes_17,...,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
ID,,,,,,,,,,,,,,,,,,,,,
5001718,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5001719,False,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
5001720,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5001723,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5001726,False,False,True,True,True,True,True,True,True,True,...,True,True,True,False,False,False,True,True,True,True


In [78]:
# 2) Seu código aqui - Marque para cada cliente se ele teve pelo menos um episódio de default entre o mês 0 e o mês 24. Dica: o método sum() pode ajudar. Caso precise, consulte a documentação e procure pelo argumento axis, você viu outros métodos que possuem esse argumento também

pg_indicadora_default.sum(axis=1) # aplicando sum() a partir do eixo das colunas é possível encontrar os clientes onde default é True

ID
5001718     0
5001719    23
5001720     0
5001723     0
5001726    19
           ..
5150475    25
5150476     0
5150480    19
5150482     6
5150487    25
Length: 20937, dtype: int64

In [79]:
pg_indicadora_default.loc[pg_indicadora_default.sum(axis=1) >= 1, 'default'] = True # a partir de 1 mês em default recebem True
pg_indicadora_default.loc[pg_indicadora_default.sum(axis=1) == 0, 'default'] = False # se igual a zero, False

In [80]:
pg_indicadora_default.head()

,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,mes_17,...,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9,default
ID,,,,,,,,,,,,,,,,,,,,,
5001718,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5001719,False,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
5001720,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5001723,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5001726,False,False,True,True,True,True,True,True,True,True,...,True,True,False,False,False,True,True,True,True,True


In [82]:
pg_default = pg_indicadora_default.default
pg_default

ID
5001718    False
5001719     True
5001720    False
5001723    False
5001726     True
           ...  
5150475     True
5150476    False
5150480     True
5150482     True
5150487     True
Name: default, Length: 20937, dtype: object

In [98]:
# 3 e 4) Seu código aqui - Faça uma junção das informações da base de propostas com a variável de default que você acabou de construir. Talvez você consiga realizar a tarefa 3 e tarefa 4 em uma única linha de código ;)
propostas = propostas.merge(pg_default, on = 'ID', how='right')
propostas

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,default_x,default_y
ID,,,,,,,,,,,,,,,,,,,
5001718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
5001719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True
5001720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
5001723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
5001726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5150475,M,Y,Y,0.0,202500.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-10886.0,-1315.0,1.0,1.0,0.0,0.0,Laborers,2.0,True,True
5150476,M,Y,Y,0.0,202500.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-10886.0,-1315.0,1.0,1.0,0.0,0.0,Laborers,2.0,False,False
5150480,F,Y,Y,1.0,157500.0,Working,Secondary / secondary special,Married,House / apartment,-10808.0,-1739.0,1.0,0.0,0.0,0.0,Core staff,3.0,True,True


In [115]:
# 5) Seu código aqui - Faça uma contagem dos valores do default que você construiu.

propostas['default'].value_counts()

True     13715
False     7222
Name: default, dtype: int64